In [ ]:
# -*- coding: utf-8 -*-
# Import packages

# Database and cloud storage
import psycopg2
import gcsfs
from google.oauth2 import service_account
from google.cloud import storage

# Asynchronous operations
import aiohttp
import ssl
import certifi
import asyncio
import nest_asyncio

# Data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

# Machine learning and preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, label_binarize
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, cross_val_predict
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_curve, auc, roc_auc_score

# Machine learning models
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Deep learning with TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier


# Loding Data 

# Loding Data 

In [ ]:
# Read the merged CSV file
merged_file_path = '/Users/tyh/Desktop/Coding/thesis/data/thesis_aki_patients_classci.csv'
df = pd.read_csv(merged_file_path)

df.head()

In [ ]:
df.info()

# survival risk category

In [ ]:
def categorize_survival_days(days):
    if days <= 200:
        return '0'  #Short-term survival
    elif 200 < days <= 800:
        return '1'  #Medium-term survival
    elif 800 < days <= 1500:
        return '2'  #Long-term survival
    else:
        return '3'  #Very long-term survival
    
# the categorize of  survival_days 
df['survival_category'] = df['survival_days'].apply(categorize_survival_days)

In [ ]:
# Calculate the number of each category in survival_category
compute_df_category_counts = df['survival_category'].value_counts()

# Calculation ratio
compute_df_total_samples = len(df)
compute_df_category_proportions = compute_df_category_counts / compute_df_total_samples

print(compute_df_category_counts)
print(compute_df_category_proportions)

# Drop the columns that are not needed

In [ ]:
df_extracted = df.drop(columns=['confirm_aki_time', 'final_deathtime'])
df_extracted

### check missing value

In [ ]:
# missing values
missing_values = df_extracted.isnull().sum()
print("the num of missing_values:")
print(missing_values)

# Prcessing the missing values

#### Firstly， Remove columns with all values as missing values, then to ensure data robustness, patients are deleted if more than 50 per cent of their data is missing

In [ ]:
 # Remove all columns with missing values
df_cleaned = df_extracted.dropna(axis=1, how='all')

# print the shape of the cleaned DataFrame
print("Shape after removal of all empty columns:", df_cleaned.shape)

#### Secondly, set threshold = 0.5 for missing values

In [ ]:
# Set the threshold for missing values
threshold = 0.5  # Set the threshold for missing values
df_cleaned_new = df_cleaned.dropna(thresh=int(threshold * df_cleaned.shape[1]))

df_cleaned_new

In [ ]:
df_cleaned_new.info()

# Completing missing values，using MICE to fill

In [ ]:
# Select the value column
numeric_columns = df_cleaned_new.select_dtypes(include=[float, int]).columns
# Using MICE to Fill in Missing Values in Value Columns
mice_imputer = IterativeImputer(max_iter=10, random_state=42, imputation_order='ascending', initial_strategy='mean')

df_numeric_imputed = pd.DataFrame(mice_imputer.fit_transform(df_cleaned_new[numeric_columns]), columns=numeric_columns)

df_numeric_imputed

In [ ]:
# Check if there are still missing values in the filled DataFrame
final_missing_values = df_numeric_imputed.isnull().sum()
print("Missing value statistics after filling:")
print(final_missing_values)

# Data Exploration

In [ ]:
df_numeric_imputed.describe()

### Defination the categorize of survival_days 

In [ ]:
def categorize_survival_days(days):
    if days <= 200:
        return '0'  #Short-term survival
    elif 200 < days <= 800:
        return '1'  #Medium-term survival
    elif 800 < days <= 1500:
        return '2'  #Long-term survival
    else:
        return '3'  #Very long-term survival
    
# the categorize of  survival_days 
df_numeric_imputed['survival_category'] = df_numeric_imputed['survival_days'].apply(categorize_survival_days)

df_numeric_imputed.head()

In [ ]:
# Calculate the number of each category in survival_category
category_counts = df_numeric_imputed['survival_category'].value_counts()

# Calculation ratio
total_samples = len(df)
category_proportions = category_counts / total_samples

print(category_counts)
print(category_proportions)

### Normalisation, min - max

In [ ]:
# min
columns_to_normalize = [
    'bmi', 
    'after_aki_creatinine_max', 'after_aki_creatinine_min',
    'after_aki_glucose_max', 'after_aki_glucose_min',
    'after_aki_potassium_max', 'after_aki_potassium_min',
    'after_aki_platelet_max', 'after_aki_platelet_min',
    'after_aki_sp_o2_max', 'after_aki_sp_o2_min',
    'creatinine_max', 'creatinine_min',
    'glucose_max', 'glucose_min',
    'potassium_max', 'potassium_min',
    'platelet_max', 'platelet_min',
    'sp_o2_max', 'sp_o2_min'
]

# 初始化 MinMaxScaler
scaler = MinMaxScaler()

# 对指定列进行归一化，结果存储在一个新的 DataFrame 中
df_final = df_numeric_imputed.copy()
df_final[columns_to_normalize] = scaler.fit_transform(df_numeric_imputed[columns_to_normalize])

df_final

In [ ]:
df_numeric_imputed

### Data imbalance exists, oversampling using SMOTE, and split the dataset into training and test sets, finally, perform feature standardisation

In [ ]:
# Selection of features and target variables
features = [
    'gender','bmi','aki_age', 
    'after_aki_creatinine_max', 'after_aki_creatinine_min',
    'after_aki_glucose_max', 'after_aki_glucose_min',
    'after_aki_potassium_max', 'after_aki_potassium_min',
    'after_aki_platelet_max', 'after_aki_platelet_min',
    'after_aki_sp_o2_max', 'after_aki_sp_o2_min',
    'creatinine_max', 'creatinine_min',
    'glucose_max', 'glucose_min',
    'potassium_max', 'potassium_min',
    'platelet_max', 'platelet_min',
    'sp_o2_max', 'sp_o2_min'
    ]
    
X = df_final[features]
y = df_final['survival_category']

# Encoding the target variable as a number
y = y.astype('category').cat.codes

# oversampling using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Perform feature standardisation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# LR Model Part 

## Fit model

In [ ]:
# Create logistic regression model and train it
lr_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)
lr_model.fit(X_train_scaled, y_train)

# Predictions on test sets
y_pred_lr = lr_model.predict(X_test_scaled)

# Calculating Accuracy, Precision, Recall and F1 Scores
accuracy = accuracy_score(y_test, y_pred_lr)
precision = precision_score(y_test, y_pred_lr, average='weighted')
recall = recall_score(y_test, y_pred_lr, average='weighted')
f1 = f1_score(y_test, y_pred_lr, average='weighted')

# print Performance Score
print("Model Performance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# print Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_lr))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr, digits=4)) # set the number of decimal places to 4

## 10-fold cross validation and GridSearch

In [ ]:
# Define the parameter grid for the logistic regression model
lr_param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # 正则化参数
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],  # 求解器
    'max_iter': [100, 200, 300]  # 最大迭代次数
}

# Creating a logistic regressionl model
lr = LogisticRegression(multi_class='multinomial', random_state=42)

# Setting Grid Search Parameters
lr_grid_search = GridSearchCV(
    estimator=lr,
    param_grid=lr_param_grid,
    cv=10,  # 10折交叉验证
    scoring='accuracy',  # 使用准确率作为评价指标
    n_jobs=-1,  # 使用所有可用的CPU核心
    verbose=2
)

lr_grid_search.fit(X_train_scaled, y_train)

# print the best parameters and best model score
print("Best parameters found: ", lr_grid_search.best_params_)
print("Best cross-validation accuracy: ", lr_grid_search.best_score_)

## Using the best parameters to repeat prediction

In [ ]:
# use the best model to make predictions
lr_best_model = lr_grid_search.best_estimator_
y_pred_lr = lr_best_model.predict(X_test_scaled)

# Calculating Accuracy, Precision, Recall and F1 Scores
accuracy = accuracy_score(y_test, y_pred_lr)
precision = precision_score(y_test, y_pred_lr, average='weighted')
recall = recall_score(y_test, y_pred_lr, average='weighted')
f1 = f1_score(y_test, y_pred_lr, average='weighted')

# print Performance Score
print("\nModel Performance Metrics on Test Data:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# print Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_lr))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr, digits=4)) # set the number of decimal places to 4

### Accuracy of LR model hyperparametric combinations

In [ ]:
# import openpyxl

# the results of the grid search
lr_results = pd.DataFrame(lr_grid_search.cv_results_)

# the accuray of cross validation select hyperparametric combinations 
lr_results_acc = lr_results[['params', 'mean_test_score', 'std_test_score']]

lr_results_acc

# save the results to an Excel file to local
lr_results_acc.to_excel('lr_grid_search_results.xlsx', index=False)

print("Results saved to 'lr_grid_search_results.xlsx'")

In [ ]:
# 
lr_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200, random_state=42)

# 10-fold cross-validation to compute the accuracy of the model
cv_scores = cross_val_score(lr_model, X_train_scaled, y_train, cv=10, scoring='accuracy', n_jobs=-1)

# Calculate the average accuracy
mean_accuracy = cv_scores.mean()

# print the accuracy of each fold and the average accuracy
print(f"Per-fold accuracy for 10-fold cross validation: {cv_scores}")
print(f"Average accuracy of 10-fold cross-validation: {mean_accuracy:.4f}")

## LR - AUC-ROC Curve

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
import numpy as np

# 4 classes
n_classes = 4
y_train_binarized = label_binarize(y_train, classes=[0, 1, 2, 3])

# 10-fold cross-validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Initialize lists to store the micro-average and weighted-average AUC values
micro_aucs = []
weighted_aucs = []

for train_idx, val_idx in kfold.split(X_train_scaled, y_train):
    # Indexing with NumPy arrays to avoid Pandas indexing errors
    X_train_fold, X_val_fold = X_train_scaled[train_idx], X_train_scaled[val_idx]
    y_train_fold, y_val_fold = y_train_binarized[train_idx], y_train_binarized[val_idx]
    
    lr_model.fit(X_train_fold, y_train[train_idx])
    y_pred_fold = lr_model.predict_proba(X_val_fold)
    
    micro_auc_fold = roc_auc_score(y_val_fold, y_pred_fold, average='micro')
    weighted_auc_fold = roc_auc_score(y_val_fold, y_pred_fold, average='weighted')
    
    micro_aucs.append(micro_auc_fold)
    weighted_aucs.append(weighted_auc_fold)

# Calculate the mean and standard deviation of the AUC values
micro_auc_cv_mean = np.mean(micro_aucs)
micro_auc_cv_std = np.std(micro_aucs)
weighted_auc_cv_mean = np.mean(weighted_aucs)
weighted_auc_cv_std = np.std(weighted_aucs)

print(f"10-Fold Cross-Validation Micro-Average AUC: Mean = {micro_auc_cv_mean:.4f}, Std Dev = {micro_auc_cv_std:.4f}")
print(f"10-Fold Cross-Validation Weighted-Average AUC: Mean = {weighted_auc_cv_mean:.4f}, Std Dev = {weighted_auc_cv_std:.4f}")

In [ ]:
# Initialize lists to store the FPR and TPR values for each fold
fprs_micro = []
tprs_micro = []
fprs_weighted = []
tprs_weighted = []

# plt roc curve
plt.figure(figsize=(10, 8))

for train_idx, val_idx in kfold.split(X_train_scaled, y_train):
    X_train_fold, X_val_fold = X_train_scaled[train_idx], X_train_scaled[val_idx]
    y_train_fold, y_val_fold = y_train_binarized[train_idx], y_train_binarized[val_idx]
    
    lr_model.fit(X_train_fold, y_train[train_idx])
    y_pred_fold = lr_model.predict_proba(X_val_fold)
    
    # compute Micro-Average ROC curve
    fpr_micro, tpr_micro, _ = roc_curve(y_val_fold.ravel(), y_pred_fold.ravel())
    fprs_micro.append(fpr_micro)
    tprs_micro.append(np.interp(np.linspace(0, 1, 100), fpr_micro, tpr_micro))
    
    # compute Weighted-Average ROC curve
    fpr_weighted, tpr_weighted, _ = roc_curve(y_val_fold.ravel(), y_pred_fold.ravel())
    fprs_weighted.append(fpr_weighted)
    tprs_weighted.append(np.interp(np.linspace(0, 1, 100), fpr_weighted, tpr_weighted))

# Micro-Average ROC curve
mean_fpr_micro = np.linspace(0, 1, 100)
mean_tpr_micro = np.mean(tprs_micro, axis=0)
mean_tpr_micro[-1] = 1.0
micro_auc = auc(mean_fpr_micro, mean_tpr_micro)

plt.plot(mean_fpr_micro, mean_tpr_micro, color='blue', lw=4,  # 
         label=f'Micro-Average ROC (AUC = {micro_auc_cv_mean:.4f} ± {micro_auc_cv_std:.4f})')

# Weighted-Average ROC curve
mean_fpr_weighted = np.linspace(0, 1, 100)
mean_tpr_weighted = np.mean(tprs_weighted, axis=0)
mean_tpr_weighted[-1] = 1.0
weighted_auc = auc(mean_fpr_weighted, mean_tpr_weighted)

plt.plot(mean_fpr_weighted, mean_tpr_weighted, color='red', lw=2, 
         label=f'Weighted-Average ROC (AUC = {weighted_auc_cv_mean:.4f} ± {weighted_auc_cv_std:.4f})')

# 45-degree line
plt.plot([0, 1], [0, 1], linestyle='--', color='grey', lw=2)

# drop the grid
plt.grid(False)

# set the x and y axis limits
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=15)
plt.ylabel('True Positive Rate', fontsize=15)
plt.title('ROC Curve for 10-Fold Cross-Validation', fontsize=18)
plt.legend(loc="lower right", fontsize=12)

# print plot
plt.show()

## the best parameters AUC

In [ ]:
# Binarize the test labels for AUC calculation
y_test_binarized = label_binarize(y_test, classes=[0, 1, 2, 3])

# Fit the best model on the training data
y_pred_best = lr_best_model.predict_proba(X_test_scaled)

# Compute the Micro-Average AUC and Weighted-Average AUC
n_bootstrap = 1000
micro_aucs_bootstrap = []
weighted_aucs_bootstrap = []

for i in range(n_bootstrap):
    indices = np.random.choice(np.arange(len(y_test)), size=len(y_test), replace=True)
    y_test_sample = y_test_binarized[indices]
    y_pred_sample = y_pred_best[indices]
    
    micro_auc_bootstrap = roc_auc_score(y_test_sample, y_pred_sample, average='micro')
    weighted_auc_bootstrap = roc_auc_score(y_test_sample, y_pred_sample, average='weighted')
    
    micro_aucs_bootstrap.append(micro_auc_bootstrap)
    weighted_aucs_bootstrap.append(weighted_auc_bootstrap)

# Calculate the mean and standard deviation of the AUC values
micro_auc_best_mean = np.mean(micro_aucs_bootstrap)
micro_auc_best_std = np.std(micro_aucs_bootstrap)
weighted_auc_best_mean = np.mean(weighted_aucs_bootstrap)
weighted_auc_best_std = np.std(weighted_aucs_bootstrap)

print(f"Best Model Micro-Average AUC: Mean = {micro_auc_best_mean:.4f}, Std Dev = {micro_auc_best_std:.4f}")
print(f"Best Model Weighted-Average AUC: Mean = {weighted_auc_best_mean:.4f}, Std Dev = {weighted_auc_best_std:.4f}")

In [ ]:
# Binarize the test labels for AUC calculation
y_test_binarized = label_binarize(y_test, classes=[0, 1, 2, 3])

# Fit the best model on the training data
y_pred_best = lr_best_model.predict_proba(X_test_scaled)

# compute Micro-Average ROC curve
fpr_micro, tpr_micro, _ = roc_curve(y_test_binarized.ravel(), y_pred_best.ravel())
micro_auc = auc(fpr_micro, tpr_micro)

# compute Weighted-Average ROC curve
fpr_weighted, tpr_weighted, _ = roc_curve(y_test_binarized.ravel(), y_pred_best.ravel())
weighted_auc = auc(fpr_weighted, tpr_weighted)

# plt roc curve 
plt.figure(figsize=(10, 8))

# Micro-Average ROC curve
plt.plot(fpr_micro, tpr_micro, color='blue', lw=5, 
         label=f'Micro-Average ROC (AUC = {micro_auc_best_mean:.4f} ± {micro_auc_best_std:.4f})')

# Weighted-Average ROC curve
plt.plot(fpr_weighted, tpr_weighted, color='red', lw=2, 
         label=f'Weighted-Average ROC (AUC = {weighted_auc_best_mean:.4f} ± {weighted_auc_best_std:.4f})')

# 45-degree line
plt.plot([0, 1], [0, 1], linestyle='--', color='grey', lw=2)

# drop the grid
plt.grid(False)

# set the x and y axis limits
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=15)
plt.ylabel('True Positive Rate', fontsize=15)
plt.title('ROC Curve for Best Model on Test Data', fontsize=18)
plt.legend(loc="lower right", fontsize=12)

# plt 
plt.show()


# SVM Model Part

In [ ]:
# create svm model and fit it
svm_classifier = SVC(kernel='rbf', decision_function_shape='ovo', random_state=42)
svm_classifier.fit(X_train_scaled, y_train)

# Predictions on test sets
y_pred_svm = svm_classifier.predict(X_test_scaled)

# Calculating Accuracy, Precision, Recall and F1 Scores
accuracy = accuracy_score(y_test, y_pred_lr)
precision = precision_score(y_test, y_pred_lr, average='weighted')
recall = recall_score(y_test, y_pred_lr, average='weighted')
f1 = f1_score(y_test, y_pred_lr, average='weighted')

# print Performance Score
print("SVM Model Performance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# print Confusion Matrix and Classification Report
print("\nSVM Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_lr))

print("\nSVM Classification Report:")
print(classification_report(y_test, y_pred_lr, digits=4)) # set the number of decimal places to 4

In [ ]:
# 
svm_param_grid = {
    'C': [0.1, 1, 10, 100],        # 
    'gamma': [0.01, 0.1, 1, 10],   # 
    'kernel': ['linear', 'rbf'],    # 
    'decision_function_shape':['ovo','ovr']
}

# create a SVM model
svm_model = SVC(decision_function_shape='ovr', random_state=42)

# Setting Grid Search Parameters
svm_grid_search = GridSearchCV(
    estimator=svm_model,
    param_grid=svm_param_grid,
    cv=10,  # 
    scoring='accuracy',  # 
    n_jobs=-1,  #
    verbose=2
)

# fit the model
svm_grid_search.fit(X_train_scaled, y_train)

# print the best parameters and best model score
print("Best parameters found: ", svm_grid_search.best_params_)
print("Best cross-validation accuracy: ", svm_grid_search.best_score_)

In [ ]:
# use the best model to make predictions
best_svm_model = svm_grid_search.best_estimator_
y_pred_svm = best_svm_model.predict(X_test_scaled)

# Calculating Accuracy, Precision, Recall and F1 Scores
accuracy = accuracy_score(y_test, y_pred_svm)
precision = precision_score(y_test, y_pred_svm, average='weighted')
recall = recall_score(y_test, y_pred_svm, average='weighted')
f1 = f1_score(y_test, y_pred_svm, average='weighted')

# print Performance Score
print("SVM Model Performance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


# print Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_svm))


print("\nClassification Report:")
print(classification_report(y_test, y_pred_svm, digits=4)) # set the number of decimal places to 4

### Accuracy of svm model hyperparametric combinations

In [ ]:
svm_results = pd.DataFrame(svm_grid_search.cv_results_)

svm_results_acc = svm_results[['params', 'mean_test_score', 'std_test_score']]

svm_results_acc

svm_results_acc.to_excel('svm_grid_search_results.xlsx', index=False)

print("Results saved to 'svm_grid_search_results.xlsx'")

In [ ]:
# svm_classifier = SVC(kernel='rbf', decision_function_shape='ovo', random_state=42)
svm_classifier = SVC(decision_function_shape='ovr', random_state=42, probability=True)

# 10-fold cross-validation to compute the accuracy of the model
cv_scores = cross_val_score(svm_classifier, X_train_scaled, y_train, cv=10, scoring='accuracy', n_jobs=-1)

# Calculate the average accuracy
mean_accuracy = cv_scores.mean()

# print the accuracy of each fold and the average accuracy
print(f"Per-fold accuracy for 10-fold cross validation: {cv_scores}")
print(f"Average accuracy of 10-fold cross-validation: {mean_accuracy:.4f}")

In [ ]:
n_classes = 4
classes = [0, 1, 2, 3]

svm_classifier = SVC(kernel='rbf', decision_function_shape='ovo', random_state=42, probability=True)

# Binarize the labels for AUC calculation
y_train_bin = label_binarize(y_train, classes=classes)
y_test_bin = label_binarize(y_test, classes=classes)

# 10-fold cross-validation
# svm_classifier = SVC(kernel='rbf', decision_function_shape='ovo', probability=True, random_state=42)

# Initialize lists to store the micro-average and weighted-average AUC values
y_scores_cv = cross_val_predict(svm_classifier, X_train_scaled, y_train, cv=3, method="predict_proba", n_jobs=-1)

# compute Micro-Average AUC
micro_auc_cv = roc_auc_score(y_train_bin, y_scores_cv, average="micro")
micro_auc_cv_std = np.std(cross_val_score(svm_classifier, X_train_scaled, y_train, cv=3, scoring='roc_auc_ovr'))

# compute Weighted-Average AUC
weighted_auc_cv = roc_auc_score(y_train_bin, y_scores_cv, average="weighted")
weighted_auc_cv_std = np.std(cross_val_score(svm_classifier, X_train_scaled, y_train, cv=3, scoring='roc_auc_ovr'))

print(f"10 fold cross validation Micro-Average AUC: Mean = {micro_auc_cv:.4f}, Std Dev = {micro_auc_cv_std:.4f}")
print(f"10 fold cross validation Weighted-Average AUC: Mean = {weighted_auc_cv:.4f}, Std Dev = {weighted_auc_cv_std:.4f}")


In [ ]:
best_svm_model= SVC(C=100,kernel='rbf', decision_function_shape='ovo', gamma=0.1, random_state=42, probability=True)

# 在整个训练集上拟合最佳模型
best_svm_model.fit(X_train_scaled, y_train)

# Initialize lists to store the FPR and TPR values for each fold
y_scores_best = best_svm_model.predict_proba(X_test_scaled)

# compute Micro-Average ROC curve
micro_auc_best = roc_auc_score(y_test_bin, y_scores_best, average="micro")
micro_auc_best_std = np.std(cross_val_score(best_svm_model, X_train_scaled, y_train, cv=3, scoring='roc_auc_ovr'))

# compute Weighted-Average ROC curve
weighted_auc_best = roc_auc_score(y_test_bin, y_scores_best, average="weighted")
weighted_auc_best_std = np.std(cross_val_score(best_svm_model, X_train_scaled, y_train, cv=3, scoring='roc_auc_ovr'))

# print the AUC values
print(f"The best Micro-Average AUC: Mean = {micro_auc_best:.4f}, Std Dev = {micro_auc_best_std:.4f}")
print(f"The best Weighted-Average AUC: Mean = {weighted_auc_best:.4f}, Std Dev = {weighted_auc_best_std:.4f}")

In [ ]:
import matplotlib.pyplot as plt

micro_auc_cv=0.9238
micro_auc_cv_std=0.0030


weighted_auc_cv=0.9112
weighted_auc_cv_std=0.0030

# 绘制交叉验证的 AUC 和方差的 ROC 曲线
plt.figure(figsize=(10, 8))


# Micro-Average ROC Curve for cross-validation
plt.plot(fpr_micro_cv, tpr_micro_cv, color='blue', lw=4,
         label=f'Micro-Average ROC curve (AUC = {micro_auc_cv:.4f} ± {micro_auc_cv_std:.4f})')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('SVM Model Cross Validation Micro & Weighted-Average ROC')
plt.legend(loc="lower right")

# Weighted-Average ROC Curve for cross-validation
plt.plot(fpr_weighted_cv, tpr_weighted_cv, color='red', lw=2,
         label=f'Weighted-Average ROC curve (AUC = {weighted_auc_cv:.4f} ± {weighted_auc_cv_std:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

micro_auc_best=0.9552
micro_auc_best_std=0.0023


weighted_auc_best=0.9411
weighted_auc_best_std=0.0023

# 绘制最佳参数模型的 AUC 和方差的 ROC 曲线
plt.figure(figsize=(10, 8))
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

# Micro-Average ROC Curve for best model
plt.plot(fpr_micro_best, tpr_micro_best, color='blue', lw=4,
         label=f'Micro-Average ROC curve (AUC = {micro_auc_best:.4f} ± {micro_auc_best_std:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('SVM Best Model Micro & Weighted-Average ROC')
plt.legend(loc="lower right")

# Weighted-Average ROC Curve for best model
plt.plot(fpr_weighted_best, tpr_weighted_best, color='red', lw=2,
         label=f'Weighted-Average ROC curve (AUC = {weighted_auc_best:.4f} ± {weighted_auc_best_std:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()


# Random Forest Part

In [ ]:
# Create a random forest model and train it
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Predictions on test sets
y_pred_rf = rf_model.predict(X_test_scaled)

# Calculating Accuracy, Precision, Recall and F1 Scores
accuracy = accuracy_score(y_test, y_pred_rf)
precision = precision_score(y_test, y_pred_rf, average='weighted')
recall = recall_score(y_test, y_pred_rf, average='weighted')
f1 = f1_score(y_test, y_pred_rf, average='weighted')

# print Performance Score
print("Model Performance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# print Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf, digits=4)) # set the number of decimal places to 4

In [ ]:
# Define the parameter grid for the random forest model
rf_param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
    'bootstrap': [True, False]  # Method of selecting samples for training each tree
}

# Create a random forest model
rf_model = RandomForestClassifier(random_state=42)

# Setting Grid Search Parameters
rf_grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=rf_param_grid,
    cv=10,  # 10-fold cross-validation
    scoring='accuracy',  # Use accuracy as the evaluation metric
    n_jobs=-1,  # Use all available CPU cores
    verbose=2
)

# Fit the model
rf_history = rf_grid_search.fit(X_train_scaled, y_train)

# print the best parameters and best model score
print("Best parameters found: ", rf_grid_search.best_params_)
print("Best cross-validation accuracy: ", rf_grid_search.best_score_)

### Accuracy of RF model hyperparametric combinations

In [ ]:
rf_results = pd.DataFrame(rf_grid_search.cv_results_)

rf_results_acc = rf_results[['params', 'mean_test_score', 'std_test_score']]

rf_results_acc

rf_results_acc.to_excel('rf_grid_search_results.xlsx', index=False)

print("Results saved to 'rf_grid_search_results.xlsx'")

In [ ]:
rf_model = RandomForestClassifier(random_state=42)

# use the best model to make predictions
cv_scores = cross_val_score(rf_model, X_train_scaled, y_train, cv=10, scoring='accuracy', n_jobs=-1)

# Calculate the average accuracy
mean_accuracy = cv_scores.mean()

# print the accuracy of each fold and the average accuracy
print(f"Per-fold accuracy for 10-fold cross validation: {cv_scores}")
print(f"Average accuracy of 10-fold cross-validation: {mean_accuracy:.4f}")

In [ ]:
# use the best model to make predictions
best_model_rf = rf_grid_search.best_estimator_
y_pred_rf = best_model_rf.predict(X_test_scaled)

# Calculating Accuracy, Precision, Recall and F1 Scores
accuracy = accuracy_score(y_test, y_pred_rf)
precision = precision_score(y_test, y_pred_rf, average='weighted')
recall = recall_score(y_test, y_pred_rf, average='weighted')
f1 = f1_score(y_test, y_pred_rf, average='weighted')

# print Performance Score
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))

# print Performance Score
print("Model Performance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf, digits=4)) # set the number of decimal places to 4

In [ ]:
classes = [0, 1, 2, 3]
y_train_bin = label_binarize(y_train, classes=classes)
y_test_bin = label_binarize(y_test, classes=classes)

# 10-fold cross-validation
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
y_scores_cv = cross_val_predict(rf_model, X_train_scaled, y_train, cv=10, method="predict_proba", n_jobs=-1)

# compute Micro-Average AUC
micro_auc_cv = roc_auc_score(y_train_bin, y_scores_cv, average="micro")
micro_auc_cv_std = np.std(cross_val_score(rf_model, X_train_scaled, y_train, cv=10, scoring='roc_auc_ovr'))

# compute Weighted-Average AUC
weighted_auc_cv = roc_auc_score(y_train_bin, y_scores_cv, average="weighted")
weighted_auc_cv_std = np.std(cross_val_score(rf_model, X_train_scaled, y_train, cv=10, scoring='roc_auc_ovr'))

print(f"10fold cross validation Micro-Average AUC: Mean = {micro_auc_cv:.4f}, Std Dev = {micro_auc_cv_std:.4f}")
print(f"10fold cross validation Weighted-Average AUC: Mean = {weighted_auc_cv:.4f}, Std Dev = {weighted_auc_cv_std:.4f}")

In [ ]:
# best model 
best_model_rf= RandomForestClassifier(n_estimators=200, max_depth=30, min_samples_leaf=1, min_samples_split=2, bootstrap=False, random_state=42)

best_model_rf.fit(X_train_scaled, y_train)

y_scores_best = best_model_rf.predict_proba(X_test_scaled)

# compute Micro-Average ROC curve
micro_auc_best = roc_auc_score(y_test_bin, y_scores_best, average="micro")
micro_auc_best_std = np.std(cross_val_score(best_model_rf, X_train_scaled, y_train, cv=10, scoring='roc_auc_ovr'))

# compute Weighted-Average ROC curve
weighted_auc_best = roc_auc_score(y_test_bin, y_scores_best, average="weighted")
weighted_auc_best_std = np.std(cross_val_score(best_model_rf, X_train_scaled, y_train, cv=10, scoring='roc_auc_ovr'))

print(f"The best Micro-Average AUC: Mean = {micro_auc_best:.4f}, Std Dev = {micro_auc_best_std:.4f}")
print(f"The best Weighted-Average AUC: Mean = {weighted_auc_best:.4f}, Std Dev = {weighted_auc_best_std:.4f}")

In [ ]:
import matplotlib.pyplot as plt

micro_auc_cv=0.9736
micro_auc_cv_std=0.0010
weighted_auc_cv=0.9685
weighted_auc_cv_std=0.0010


# 绘制交叉验证的 AUC 和方差的 ROC 曲线
plt.figure(figsize=(10, 8))
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

# Micro-Average ROC Curve for cross-validation
plt.plot(fpr_micro_cv, tpr_micro_cv, color='blue', lw=4,
         label=f'Micro-Average ROC curve (AUC = {micro_auc_cv:.4f} ± {micro_auc_cv_std:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('RF Model Cross Validation Micro & Weighted-Average ROC')
plt.legend(loc="lower right")

# Weighted-Average ROC Curve for cross-validation
plt.plot(fpr_weighted_cv, tpr_weighted_cv, color='red', lw=2,
         label=f'Weighted-Average ROC curve (AUC = {weighted_auc_cv:.4f} ± {weighted_auc_cv_std:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

micro_auc_best=0.9802
micro_auc_best_std=0.0010

weighted_auc_best=0.9756
weighted_auc_best_std=0.0010


# 绘制最佳参数模型的 AUC 和方差的 ROC 曲线
plt.figure(figsize=(10, 8))
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

# Micro-Average ROC Curve for best model
plt.plot(fpr_micro_best, tpr_micro_best, color='blue', lw=4,
         label=f'Micro-Average ROC curve (AUC = {micro_auc_best:.4f} ± {micro_auc_best_std:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('RF Best Model Micro & Weighted-Average ROC')
plt.legend(loc="lower right")

# Weighted-Average ROC Curve for best model
plt.plot(fpr_weighted_best, tpr_weighted_best, color='red', lw=2,
         label=f'Weighted-Average ROC curve (AUC = {weighted_auc_best:.4f} ± {weighted_auc_best_std:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

# XGBoost & LightGBM Model Part 

### XGBosot

In [ ]:
# Create a XGBoost model and train it
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train_scaled, y_train)

# Predictions on test sets
y_pred_xgb = xgb_model.predict(X_test_scaled)

# Calculating Accuracy, Precision, Recall and F1 Scores
accuracy = accuracy_score(y_test, y_pred_xgb)
precision = precision_score(y_test, y_pred_xgb, average='weighted')
recall = recall_score(y_test, y_pred_xgb, average='weighted')
f1 = f1_score(y_test, y_pred_xgb, average='weighted')

# print Performance 
print("Model Performance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# print Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_xgb))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_xgb, digits=4)) # set the number of decimal places to 4

In [ ]:
# from sklearn.model_selection import GridSearchCV, KFold
# from xgboost import XGBClassifier
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# create a XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Define the parameter grid for the XGBoost model
xgb_param_grid = {
    'n_estimators': [100, 200, 300],    # Number of trees
    'max_depth': [3, 5, 7],             # Maximum depth of the tree
    'learning_rate': [0.01, 0.1, 0.2],  # Learning rate
    'subsample': [0.8, 1.0],            # Subsample ratio of the training instances
    'colsample_bytree': [0.8, 1.0]      # Subsample ratio of columns when constructing each tree
}

# Create a 10-fold cross-validation object
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Create a Grid Search object
grid_search = GridSearchCV(estimator=xgb_model, param_grid=xgb_param_grid, cv=cv, scoring='accuracy', n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(X_train_scaled, y_train)

# print the best parameters and best model score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

# use the best model to make predictions
best_xgb_model = grid_search.best_estimator_
y_pred_xgb = best_xgb_model.predict(X_test_scaled)

# Calculating Accuracy, Precision, Recall and F1 Scores
accuracy = accuracy_score(y_test, y_pred_xgb)
precision = precision_score(y_test, y_pred_xgb, average='weighted')
recall = recall_score(y_test, y_pred_xgb, average='weighted')
f1 = f1_score(y_test, y_pred_xgb, average='weighted')

# print Performance
print("\nModel Performance Metrics on Test Data:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# print Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_xgb))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_xgb, digits=4))

### Accuracy of XGBoost model hyperparametric combinations

In [ ]:
xgboost_results = pd.DataFrame(grid_search.cv_results_)

xgboost_results_acc = xgboost_results[['params', 'mean_test_score', 'std_test_score']]

xgboost_results_acc

xgboost_results_acc.to_excel('xbg_grid_search_results.xlsx', index=False)

print("Results saved to 'xbg_grid_search_results.xlsx'")

In [ ]:
xgb_model = XGBClassifier(colsample_bytree=1.0,learning_rate=0.2, max_depth=7, n_estimators=300, subsample=0.8, random_state=42)
# use the best model to make predictions
cv_scores = cross_val_score(xgb_model, X_train_scaled, y_train, cv=10, scoring='accuracy', n_jobs=-1)

# Calculate the average accuracy
mean_accuracy = cv_scores.mean()

# print the accuracy of each fold and the average accuracy
print(f"Per-fold accuracy for 10-fold cross validation: {cv_scores}")
print(f"Average accuracy of 10-fold cross-validation: {mean_accuracy:.4f}")

### AUC for XGBoost

In [ ]:
classes = [0, 1, 2, 3]
y_train_bin = label_binarize(y_train, classes=classes)
y_test_bin = label_binarize(y_test, classes=classes)

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

y_scores_cv = cross_val_predict(xgb_model, X_train_scaled, y_train, cv=10, method="predict_proba", n_jobs=-1)

# compute Micro-Average AUC
micro_auc_cv = roc_auc_score(y_train_bin, y_scores_cv, average="micro")
micro_auc_cv_std = np.std(cross_val_score(xgb_model, X_train_scaled, y_train, cv=10, scoring='roc_auc_ovr'))

# 计算Weighted-Average AUC
weighted_auc_cv = roc_auc_score(y_train_bin, y_scores_cv, average="weighted")
weighted_auc_cv_std = np.std(cross_val_score(xgb_model, X_train_scaled, y_train, cv=10, scoring='roc_auc_ovr'))

print(f"10fold cross validation Micro-Average AUC: Mean = {micro_auc_cv:.4f}, Std Dev = {micro_auc_cv_std:.4f}")
print(f"10fold cross validation Weighted-Average AUC: Mean = {weighted_auc_cv:.4f}, Std Dev = {weighted_auc_cv_std:.4f}")

In [ ]:
best_xgb_model = XGBClassifier(colsample_bytree=1.0,learning_rate=0.2, max_depth=7, n_estimators=300, subsample=0.8, random_state=42)

best_xgb_model.fit(X_train_scaled, y_train)

# compute the auc for the best model
y_scores_best = best_xgb_model.predict_proba(X_test_scaled)

# compute Micro-Average ROC curve
micro_auc_best = roc_auc_score(y_test_bin, y_scores_best, average="micro")
micro_auc_best_std = np.std(cross_val_score(best_xgb_model, X_train_scaled, y_train, cv=10, scoring='roc_auc_ovr'))

# compute Weighted-Average ROC curve
weighted_auc_best = roc_auc_score(y_test_bin, y_scores_best, average="weighted")
weighted_auc_best_std = np.std(cross_val_score(best_xgb_model, X_train_scaled, y_train, cv=10, scoring='roc_auc_ovr'))

print(f"The best Micro-Average AUC: Mean = {micro_auc_best:.4f}, Std Dev = {micro_auc_best_std:.4f}")
print(f"The best Weighted-Average AUC: Mean = {weighted_auc_best:.4f}, Std Dev = {weighted_auc_best_std:.4f}")

In [ ]:
# plt the roc curve for 10-fold cross-validation
fpr_micro_cv, tpr_micro_cv, _ = roc_curve(y_train_bin.ravel(), y_scores_cv.ravel())
fpr_weighted_cv, tpr_weighted_cv, _ = roc_curve(y_train_bin.ravel(), y_scores_cv.ravel())

micro_auc_cv=0.9782
micro_auc_cv_std=0.0007
weighted_auc_cv=0.9745
weighted_auc_cv_std=0.0007

plt.figure(figsize=(10, 8))
plt.xlim([0.0, 1.0])

plt.ylim([0.0, 1.05])
plt.plot(fpr_micro_cv, tpr_micro_cv, color='blue', lw=4, label=f'Micro-Average ROC curve (AUC = {micro_auc_cv:.4f} ± {micro_auc_cv_std:.4f})')
plt.plot(fpr_weighted_cv, tpr_weighted_cv, color='red', lw=2, label=f'Weighted-Average ROC curve (AUC = {weighted_auc_cv:.4f} ± {weighted_auc_cv_std:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('XGBoost Model Cross Validation Micro & Weighted-Average ROC')
plt.legend(loc="lower right")
plt.show()




# plt the roc curve for the best model
fpr_micro_best, tpr_micro_best, _ = roc_curve(y_test_bin.ravel(), y_scores_best.ravel())
fpr_weighted_best, tpr_weighted_best, _ = roc_curve(y_test_bin.ravel(), y_scores_best.ravel())

micro_auc_best=0.9841
micro_auc_best_std=0.0009

weighted_auc_best=0.9803
weighted_auc_best_std=0.0009



plt.figure(figsize=(10, 8))
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.plot(fpr_micro_best, tpr_micro_best, color='blue', lw=4, label=f'Micro-Average ROC curve (AUC = {micro_auc_best:.4f} ± {micro_auc_best_std:.4f})')
plt.plot(fpr_weighted_best, tpr_weighted_best, color='red', lw=2, label=f'Weighted-Average ROC curve (AUC = {weighted_auc_best:.4f} ± {weighted_auc_best_std:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('XGBoost Best Model Micro & Weighted-Average ROC')
plt.legend(loc="lower right")
plt.show()


# plt.plot([0, 1], [0, 1], linestyle='--', color='grey', lw=2)

# # 去掉网格
# plt.grid(False)

# # 设置图形的范围、标签、标题和图例
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.grid(False)
# plt.xlabel('False Positive Rate', fontsize=15)
# plt.ylabel('True Positive Rate', fontsize=15)
# plt.title('LR Best Model Micro & Weighted-Average ROC', fontsize=18)
# plt.legend(loc="lower right", fontsize=12)

# # 显示图形
# plt.show()



# LightGBM Model Part

In [ ]:
# create lightGBM_model
lightGBM_model = LGBMClassifier(
    objective='multiclass',
    num_class=len(y.unique()),
    boosting_type='gbdt',
    learning_rate=0.1,
    n_estimators=100,
    max_depth=5,
    random_state=42
)

#  fit model
lightGBM_model.fit(X_train_scaled, y_train)

# Predictions on test sets
y_pred_lightGBM = lightGBM_model.predict(X_test_scaled)

# Calculating Accuracy, Precision, Recall and F1 Scores
accuracy = accuracy_score(y_test, y_pred_lightGBM)
precision = precision_score(y_test, y_pred_lightGBM, average='weighted')
recall = recall_score(y_test, y_pred_lightGBM, average='weighted')
f1 = f1_score(y_test, y_pred_lightGBM, average='weighted')

# print Performance Score
print("Model Performance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# print Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_lightGBM))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_lightGBM, digits=4)) # set the number of decimal places to 4


In [ ]:
# Define the parameter grid for the LightGBM model
lightGBM_param_grid = {
    'boosting_type': ['gbdt', 'dart'],  # Gradient Boosting Decision Tree
    'num_leaves': [31, 50, 100],        # Maximum number of leaves in one tree
    'max_depth': [-1, 10, 20],          # Maximum depth of the tree
    'learning_rate': [0.001, 0.01, 0.1, 0.2],  # 
    'n_estimators': [100, 200, 300],    # Number of trees
    'objective': ['multiclass'],        # multiclass
    'num_class': [4],                   # Number of classes
}

# create a LightGBM model
lightGBM_model = LGBMClassifier(random_state=42)

# Setting Grid Search Parameters
lightGBM_grid_search = GridSearchCV(
    estimator=lightGBM_model,
    param_grid=lightGBM_param_grid,
    cv=10,  # 
    scoring='accuracy',  # 
    n_jobs=-1,  # 
    verbose=2
)

# fit the model
lightGBM_grid_search.fit(X_train_scaled, y_train)

# print the best parameters and best model score
print("Best parameters found: ", lightGBM_grid_search.best_params_)
print("Best cross-validation accuracy: ", lightGBM_grid_search.best_score_)

In [ ]:
# use the best model to make predictions
best_lightGBM_model = lightGBM_grid_search.best_estimator_
y_pred_lightGBM = best_lightGBM_model.predict(X_test_scaled)

# Calculating Accuracy, Precision, Recall and F1 Scores
accuracy = accuracy_score(y_test, y_pred_lightGBM)
precision = precision_score(y_test, y_pred_lightGBM, average='weighted')
recall = recall_score(y_test, y_pred_lightGBM, average='weighted')
f1 = f1_score(y_test, y_pred_lightGBM, average='weighted')

# print Performance Score
print("\nModel Performance Metrics on Test Data:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# print Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_lightGBM))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_lightGBM, digits=4)) # set the number of decimal places to 4

In [ ]:
classes = [0, 1, 2, 3]
y_train_bin = label_binarize(y_train, classes=classes)
y_test_bin = label_binarize(y_test, classes=classes)

lightGBM_model = LGBMClassifier(
    objective='multiclass',
    num_class=4,
    boosting_type='gbdt',
    learning_rate=0.1,
    n_estimators=100,
    max_depth=5,
    random_state=42
)

y_scores_cv = cross_val_predict(lightGBM_model, X_train_scaled, y_train, cv=10, method="predict_proba", n_jobs=-1)

# compute Micro-Average AUC
micro_auc_cv = roc_auc_score(y_train_bin, y_scores_cv, average="micro")
micro_auc_cv_std = np.std(cross_val_score(xgb_model, X_train_scaled, y_train, cv=10, scoring='roc_auc_ovr'))

# 计算Weighted-Average AUC
weighted_auc_cv = roc_auc_score(y_train_bin, y_scores_cv, average="weighted")
weighted_auc_cv_std = np.std(cross_val_score(xgb_model, X_train_scaled, y_train, cv=10, scoring='roc_auc_ovr'))

print(f"10fold cross validation Micro-Average AUC: Mean = {micro_auc_cv:.4f}, Std Dev = {micro_auc_cv_std:.4f}")
print(f"10fold cross validation Weighted-Average AUC: Mean = {weighted_auc_cv:.4f}, Std Dev = {weighted_auc_cv_std:.4f}")

In [ ]:
best_lightgbm_model = LGBMClassifier(objective='multiclass',
    num_class=4,
    boosting_type='gbdt',
    learning_rate=0.2,
    n_estimators=300,
    max_depth= -1,
    n_leaves=100,
    random_state=42)

best_lightgbm_model.fit(X_train_scaled, y_train)

# compute the auc for the best model
y_scores_best = best_lightgbm_model.predict_proba(X_test_scaled)

# compute Micro-Average ROC curve
micro_auc_best = roc_auc_score(y_test_bin, y_scores_best, average="micro")
micro_auc_best_std = np.std(cross_val_score(best_lightgbm_model, X_train_scaled, y_train, cv=10, scoring='roc_auc_ovr'))

# compute Weighted-Average ROC curve
weighted_auc_best = roc_auc_score(y_test_bin, y_scores_best, average="weighted")
weighted_auc_best_std = np.std(cross_val_score(best_lightgbm_model, X_train_scaled, y_train, cv=10, scoring='roc_auc_ovr'))

print(f"The best Micro-Average AUC: Mean = {micro_auc_best:.4f}, Std Dev = {micro_auc_best_std:.4f}")
print(f"The best Weighted-Average AUC: Mean = {weighted_auc_best:.4f}, Std Dev = {weighted_auc_best_std:.4f}")

In [ ]:
# plt the roc curve for 10-fold cross-validation
fpr_micro_cv, tpr_micro_cv, _ = roc_curve(y_train_bin.ravel(), y_scores_cv.ravel())
fpr_weighted_cv, tpr_weighted_cv, _ = roc_curve(y_train_bin.ravel(), y_scores_cv.ravel())

plt.figure(figsize=(10, 8))
plt.plot(fpr_micro_cv, tpr_micro_cv, color='blue', lw=4, label=f'Micro-Average ROC curve (AUC = {micro_auc_cv:.4f} ± {micro_auc_cv_std:.4f})')
plt.plot(fpr_weighted_cv, tpr_weighted_cv, color='red', lw=2, label=f'Weighted-Average ROC curve (AUC = {weighted_auc_cv:.4f} ± {weighted_auc_cv_std:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('LightGBM Model Cross Validation Micro & Weighted-Average ROC')
plt.legend(loc="lower right")
plt.show()

# plt the roc curve for the best model
fpr_micro_best, tpr_micro_best, _ = roc_curve(y_test_bin.ravel(), y_scores_best.ravel())
fpr_weighted_best, tpr_weighted_best, _ = roc_curve(y_test_bin.ravel(), y_scores_best.ravel())

plt.figure(figsize=(10, 8))
plt.plot(fpr_micro_best, tpr_micro_best, color='blue', lw=4, label=f'Micro-Average ROC curve (AUC = {micro_auc_best:.4f} ± {micro_auc_best_std:.4f})')
plt.plot(fpr_weighted_best, tpr_weighted_best, color='red', lw=2, label=f'Weighted-Average ROC curve (AUC = {weighted_auc_best:.4f} ± {weighted_auc_best_std:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('LightGBM Best Model Micro & Weighted-Average ROC')
plt.legend(loc="lower right")
plt.show()

### Accuracy of LightGBM model hyperparametric combinations

In [ ]:
lightGBM_results = pd.DataFrame(lightGBM_grid_search.cv_results_)

lightgbm_results_acc = lightGBM_results[['params', 'mean_test_score', 'std_test_score']]

lightgbm_results_acc

lightgbm_results_acc.to_excel('lightgbm_grid_search_results.xlsx', index=False)

print("Results saved to 'lightgbm_grid_search_results.xlsx'")

In [ ]:
lightGBM_model = LGBMClassifier(
    boosting_type='gbdt', learning_rate=0.2, max_depth= -1, 
    n_estimators=300, num_class=4, num_leaves=100, objective='multiclass', random_state=42)

# 'boosting_type': 'gbdt', 'learning_rate': 0.2, 'max_depth': -1, 'n_estimators': 300, 'num_class': 4, 'num_leaves': 100, 'objective': 'multiclass
# use the best model to make predictions
cv_scores = cross_val_score(lightGBM_model, X_train_scaled, y_train, cv=10, scoring='accuracy', n_jobs=-1)

# Calculate the average accuracy
mean_accuracy = cv_scores.mean()

# print the accuracy of each fold and the average accuracy
print(f"Per-fold accuracy for 10-fold cross validation: {cv_scores}")
print(f"Average accuracy of 10-fold cross-validation: {mean_accuracy:.4f}")

# CNN Model Part 

In [ ]:
# Separate features and target variables
X_cnn = df_numeric_imputed.drop('survival_category', axis=1).values
y_cnn = df_numeric_imputed['survival_category'].values

# standardize the features
scaler = StandardScaler()
X_scaled_cnn = scaler.fit_transform(X_cnn)

# one-hot encode the target variable
y_categorical_cnn = to_categorical(y_cnn)

# Split the data into training and testing sets
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(X_scaled_cnn, y_categorical_cnn, test_size=0.2, random_state=42)

# check the shape of the training and testing sets
print(f"training num: {X_train_cnn.shape[0]}, features: {X_train_cnn.shape[1]}")
print(f"testing num: {X_test_cnn.shape[0]}, features: {X_test_cnn.shape[1]}")
print(f"training num: {y_train_cnn.shape[0]}, features: {y_train_cnn.shape[1]}")
print(f"testing num: {y_test_cnn.shape[0]}, features: {y_test_cnn.shape[1]}")

In [ ]:
from keras.regularizers import l1_l2

# 调整输入数据的形状以适应Conv1D
X_train_cnn = X_train_cnn.reshape((X_train_cnn.shape[0], X_train_cnn.shape[1], 1))
X_test_cnn = X_test_cnn.reshape((X_test_cnn.shape[0], X_test_cnn.shape[1], 1))


cnn_model = Sequential()

# because of the data, using Conv1D
cnn_model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Conv1D(64, kernel_size=3, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))) # add regularization
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(y_categorical_cnn.shape[1], activation='softmax'))

# compile the model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# fit model 
X_train_cnn = X_train_cnn.reshape((X_train_cnn.shape[0], X_train_cnn.shape[1], 1))  # 添加一个维度用于Conv1D
X_test_cnn = X_test_cnn.reshape((X_test_cnn.shape[0], X_test_cnn.shape[1], 1))

# set early_stopping
cnn_early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
cnn_model.fit(X_train_cnn, y_train_cnn, epochs=50, batch_size=32, 
                          validation_data=(X_test_cnn, y_test_cnn), 
                          callbacks=[cnn_early_stopping], verbose=2)

# evaluate the model
loss, accuracy = cnn_model.evaluate(X_test_cnn, y_test_cnn, verbose=0)
print(f"the accuracy of testing set: {accuracy:.4f}")

# plot the training history
y_pred_cnn = cnn_model.predict(X_test_cnn)
y_pred_classes = np.argmax(y_pred_cnn, axis=1)
y_true_classes = np.argmax(y_test_cnn, axis=1)

# calculate the performance metrics
accuracy = accuracy_score(y_true_classes, y_pred_classes)
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

# print the performance metrics
print("\nModel Performance Metrics on Test Data:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

print("\nconfusion matirx:")
print(confusion_matrix(y_true_classes, y_pred_classes))

print("\nclassification_report:")
print(classification_report(y_true_classes, y_pred_classes, digits=4)) # set the number of decimal places to 4

In [ ]:
# plot the training and validation loss
plt.figure(figsize=(6, 4))
plt.plot(cnn_history.history['loss'], label='Training Loss')
plt.plot(cnn_history.history['val_loss'], label='Validation Loss')
plt.title('CNN Model - Loss Fuction Trend')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Adjust the shape of the input data to fit Conv1D
X_train_cnn = X_train_cnn.reshape((X_train_cnn.shape[0], X_train_cnn.shape[1], 1))
X_test_cnn = X_test_cnn.reshape((X_test_cnn.shape[0], X_test_cnn.shape[1], 1))

In [ ]:

from keras.models import Model, Sequential
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout

def create_cnn_model(optimizer='adam', dropout_rate=0.5):
    input_layer = Input(shape=(X_cnn.shape[1], 1))
    x = Conv1D(32, kernel_size=3, activation='relu')(input_layer)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(64, kernel_size=3, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(x)
    x = Dropout(dropout_rate)(x)
    output_layer = Dense(4, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Wrapping Models with Scikeras' KerasClassifier
cnn_classifier = KerasClassifier(model=create_cnn_model, epochs=10, batch_size=32, verbose=2)

# define param_grid
param_grid = {
    'model__optimizer': ['adam', 'rmsprop'],
    'model__dropout_rate': [0.3, 0.5, 0.7],
    'batch_size': [32, 64, 128],
    'epochs': [20, 30, 50]
}

# 
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

#  set Grid Search
cnn_grid_search = GridSearchCV(estimator=cnn_classifier, param_grid=param_grid, cv=kfold, n_jobs=-1, verbose=2)

# fit the model by Grid Search
cnn_grid_search.fit(X_scaled_cnn.reshape((X_scaled_cnn.shape[0], X_scaled_cnn.shape[1], 1)), y_categorical_cnn, verbose=1)


# print the best parameters and best model score
print("Best parameters found: ", cnn_grid_search.best_params_)
print("Best cross-validation accuracy: ", cnn_grid_search.best_score_)

# use the best model to make predictions
best_cnn_model = cnn_grid_search.best_estimator_
y_pred_cnn = best_cnn_model.predict(X_test_cnn)

# compute the confusion matrix and classification report
y_pred_classes = np.argmax(y_pred_cnn, axis=1)
y_true_classes = np.argmax(y_test_cnn, axis=1)

# calculate the performance metrics
accuracy = accuracy_score(y_true_classes, y_pred_classes)
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

# print Performance 
print("\nModel Performance Metrics on Test Data:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

print("\n confusion_matrix:")
print(confusion_matrix(y_true_classes, y_pred_classes))

print("\n classification_report:")
print(classification_report(y_true_classes, y_pred_classes, digits=4)) # set the number of decimal places to 4

In [ ]:
cnn_results = pd.DataFrame(cnn_grid_search.cv_results_)

cnn_results_acc = cnn_results[['params', 'mean_test_score', 'std_test_score']]

cnn_results_acc

cnn_results_acc.to_excel('cnn_grid_search_results.xlsx', index=False)

print("Results saved to 'cnn_grid_search_results.xlsx'")

In [ ]:
classes = [0, 1, 2, 3]

# Ensure that X_train_scaled is reshaped properly for CNN
# X_train_scaled_cnn = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))

# 训练模型
X_train_cnn = X_train_cnn.reshape((X_train_cnn.shape[0], X_train_cnn.shape[1], 1))  # 添加一个维度用于Conv1D
X_test_cnn = X_test_cnn.reshape((X_test_cnn.shape[0], X_test_cnn.shape[1], 1))

def create_cnn_model(optimizer='adam', dropout_rate=0.5):
    input_layer = Input(shape=(X_train_scaled.shape[1], 1))
    x = Conv1D(32, kernel_size=3, activation='relu')(input_layer)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(64, kernel_size=3, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(x)
    x = Dropout(dropout_rate)(x)
    output_layer = Dense(4, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


cnn_early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 确保输入数据的形状适合Conv1D
# X_train_cnn = X_scaled_cnn.reshape((X_scaled_cnn.shape[0], X_scaled_cnn.shape[1], 1))

# 定义CNN模型包装器
cnn_classifier = KerasClassifier(model=create_cnn_model, batch_size=64, epochs=30, dropout_rate=0.5, 
                                 optimizer='adam', random_state=42, verbose=2)

# 设置10折交叉验证
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# 计算交叉验证的准确率
cv_scores = cross_val_score(cnn_classifier, X_train_cnn, y_train_cnn, cv=kfold, scoring='accuracy', n_jobs=-1)

# 打印每折的准确率和平均准确率
print(f"Per-fold accuracy for 10-fold cross-validation: {cv_scores}")
print(f"Average accuracy of 10-fold cross-validation: {cv_scores.mean():.4f}")


In [ ]:
# test
X_train_cnn = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
y_train_cnn = to_categorical(y_train, num_classes=4)  # 假设4个类别

def create_cnn_model(optimizer='adam', dropout_rate=0.5):
    input_layer = Input(shape=(X_train_cnn.shape[1], 1))  # 使用X_train_cnn的形状
    x = Conv1D(32, kernel_size=3, activation='relu')(input_layer)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(64, kernel_size=3, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')#, kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(x)
    x = Dropout(dropout_rate)(x)
    output_layer = Dense(4, activation='softmax')(x)  # 假设有4个类别
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model



In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import label_binarize

n_classes = 4

# Binarize the labels for multi-class ROC AUC
y_categorical_cnn = label_binarize(y_train_cnn, classes=[0, 1, 2, 3])

# 10-Fold Cross-Validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_micro_aucs = []
cv_weighted_aucs = []
mean_fpr = np.linspace(0, 1, 100)
tprs_micro = []
tprs_weighted = []

for train_idx, val_idx in kfold.split(X_train_cnn, np.argmax(y_train_cnn, axis=1)):
    X_train_fold, X_val_fold = X_train_cnn[train_idx], X_train_cnn[val_idx]
    y_train_fold, y_val_fold = y_categorical_cnn[train_idx], y_categorical_cnn[val_idx]

    # Reshape for Conv1D input
    X_train_fold = X_train_fold.reshape((X_train_fold.shape[0], X_train_fold.shape[1], 1))
    X_val_fold = X_val_fold.reshape((X_val_fold.shape[0], X_val_fold.shape[1], 1))

    # Train the model
    best_cnn_model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=32, verbose=0)

    # Predict probabilities
    y_pred_fold = best_cnn_model.predict(X_val_fold)

    # Calculate AUC for each fold
    micro_auc = roc_auc_score(y_val_fold, y_pred_fold, average='micro')
    weighted_auc = roc_auc_score(y_val_fold, y_pred_fold, average='weighted')

    cv_micro_aucs.append(micro_auc)
    cv_weighted_aucs.append(weighted_auc)

    # Compute ROC curve and AUC for micro-average
    fpr_micro, tpr_micro, _ = roc_curve(y_val_fold.ravel(), y_pred_fold.ravel())
    tprs_micro.append(np.interp(mean_fpr, fpr_micro, tpr_micro))
    tprs_micro[-1][0] = 0.0

# Calculate mean and std deviation of the AUCs
cv_micro_auc_mean = np.mean(cv_micro_aucs)
cv_micro_auc_std = np.std(cv_micro_aucs)
cv_weighted_auc_mean = np.mean(cv_weighted_aucs)
cv_weighted_auc_std = np.std(cv_weighted_aucs)

# Plotting the results
plt.figure(figsize=(10, 8))

# Plotting the micro-average ROC curve
mean_tpr_micro = np.mean(tprs_micro, axis=0)
mean_tpr_micro[-1] = 1.0
mean_auc_micro = auc(mean_fpr, mean_tpr_micro)
plt.plot(mean_fpr, mean_tpr_micro, color='blue', linestyle='-', lw=5,
         label=f'Micro-Average ROC (AUC = {cv_micro_auc_mean:.4f} ± {cv_micro_auc_std:.4f})')

# Plotting the weighted-average ROC curve
plt.plot(mean_fpr, mean_tpr_micro, color='red', linestyle='-', lw=2,
         label=f'Weighted-Average ROC (AUC = {cv_weighted_auc_mean:.4f} ± {cv_weighted_auc_std:.4f})')

# Diagonal line (random guessing)
plt.plot([0, 1], [0, 1], 'k--', lw=2)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('10-Fold Cross-Validation ROC Curves for CNN Model')
plt.legend(loc='lower right')
# 去掉网格
plt.grid(False)

plt.show()

print(f"10-Fold Cross-Validation Micro-Average AUC: Mean = {cv_micro_auc_mean:.4f}, Std Dev = {cv_micro_auc_std:.4f}")
print(f"10-Fold Cross-Validation Weighted-Average AUC: Mean = {cv_weighted_auc_mean:.4f}, Std Dev = {cv_weighted_auc_std:.4f}")


In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize

n_classes = 4

# Binarize the test labels for AUC calculation
y_test_binarized = label_binarize(y_test_cnn, classes=[0, 1, 2, 3])

# Predict probabilities
y_pred_proba = best_cnn_model.predict_proba(X_test_cnn)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_pred_proba[:, i])
    roc_auc[i] = roc_auc_score(y_test_binarized[:, i], y_pred_proba[:, i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_binarized.ravel(), y_pred_proba.ravel())
roc_auc["micro"] = roc_auc_score(y_test_binarized, y_pred_proba, average="micro")

# Compute macro-average ROC curve and ROC area
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
mean_tpr = np.zeros_like(all_fpr)

for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i]) * np.sum(y_test_binarized[:, i])

# Average it and compute AUC
mean_tpr /= np.sum(y_test_binarized)
roc_auc["weighted"] = np.trapz(mean_tpr, all_fpr)

# Compute the standard deviation of the AUC
micro_auc_std = np.std([roc_auc[i] for i in range(n_classes)])
weighted_auc_std = np.std([roc_auc[i] for i in range(n_classes)])

# plt roc curve
plt.figure(figsize=(10, 8))

# Micro-Average ROC curve
plt.plot(fpr["micro"], tpr["micro"], label=f'Micro-Average ROC (AUC = {roc_auc["micro"]:.4f} ± {micro_auc_std:.4f})', color='blue', linestyle='-',lw=5)

# Weighted-Average ROC curve
plt.plot(all_fpr, mean_tpr, label=f'Weighted-Average ROC (AUC = {roc_auc["weighted"]:.4f} ± {weighted_auc_std:.4f})', color='red', linestyle='-',lw=2)

# Diagonal line (random guessing)
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for CNN Model')

plt.legend(loc="lower right")
plt.grid(False)  # drop the grid
plt.show()

# print the AUC values
print(f"Micro-Average AUC: {roc_auc['micro']:.4f} ± {micro_auc_std:.4f}")
print(f"Weighted-Average AUC: {roc_auc['weighted']:.4f} ± {weighted_auc_std:.4f}")
